# Project 1 – Charles – Metrics

In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib
import seaborn as sns

# Adding Session ID

In [2]:
# do not run this entire section if your dataset already has unique ID

In [3]:
#data = pd.read_csv("Cleaned_Data.csv", low_memory=False)
#df = data.sort_values(by=['distinct_id', 'time'], axis=0, ascending=True)
#df['ae_session_length'] = df['ae_session_length'].fillna('-1')
#df['ae_session_length'] = df['ae_session_length'].str.replace(',', '')
#df['ae_session_length'] = df['ae_session_length'].astype(float)
#df['time'] = df['time'] = pd.to_datetime(df['time'])
#df

In [4]:
def add_session_id(df):
    df['session_id'] = -1
    sessions = df[['name_x', 'distinct_id', 'time', 'ae_session_length']].loc[df['name_x'] == '$ae_session']
    sessions['start'] = sessions['time'] - np.array([pd.Timedelta(seconds=np.ceil(x)) for x in sessions['ae_session_length']])
    curr_id = 1
    for index, row in sessions.iterrows():
        df.loc[(df['session_id'] == -1) 
               & (df['time'] <= row['time']) 
               & (df['time'] >= row['start']) 
               & (df['distinct_id'] == row['distinct_id']), 'session_id'] = curr_id
        curr_id += 1

#add_session_id(df)
#df
    

In [5]:
#df.loc[df['session_id'] == -1]

In [6]:
#df.to_csv("Cleaned_Dataset_With_ID.csv")

# Adding Time Between Sessions

In [23]:
# do not run this entire section if your dataset already has time_deltas

In [7]:
#data = pd.read_csv("Cleaned_Dataset_With_ID.csv", low_memory=False)

In [8]:
#data.drop(columns=['Unnamed: 0'], axis=1, inplace=True)


In [9]:
#data = data.sort_values(by=['distinct_id', 'time'], axis=0, ascending=True)
#data['time'] = pd.to_datetime(data['time'])
#data

In [12]:
def add_time_delta(df):
    df['time_delta'] = 0
    grouped = df.loc[df['session_id'] != -1].groupby(['distinct_id', 'session_id'], as_index=False).agg({'time' : ['min', 'max']})
    curr_distinct_id = ""
    for index, row in grouped.iterrows():
        if row[0] != curr_distinct_id:
            curr_distinct_id = row[0]
        else:
            delta = row[2] - grouped.iloc[index - 1, 3]
            df.loc[(df['distinct_id'] == row[0]) 
                   & (df['session_id'] == row[1]), 'time_delta'] = delta

#add_time_delta(data)
#data

In [14]:
def recover(function):
    import inspect
    get_ipython().set_next_input("".join(inspect.getsourcelines(function)[0]))


In [15]:
#data.loc[data['time_delta'] == 0]

In [16]:
#data.to_csv("Cleaned_Dataset_With_ID.csv")

# Adding Slope

In [17]:
data = pd.read_csv("Cleaned_Dataset_With_ID.csv", low_memory=False)
df = data.loc[(data['name_x'] == '$ae_session') & (data['time_delta'] != 0)]
df

,Unnamed: 0,name_x,distinct_id,time,ae_session_length,city,region,mp_country_code,locationSetting,notificationSettings,...,active_timespan,average_session_time,country,US,Facebook,Foodie,Google,Unknown,session_id,time_delta
19,19,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:50:51,65.8,Palo Alto,California,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,2,64.0
21,21,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 13:51:52,59.9,Palo Alto,California,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,3,7.0
24,24,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 14:38:30,72.3,Hayward,California,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,4,2728.0
36,36,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 15:05:10,151.9,San Jose,California,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,5,1466.0
50,50,$ae_session,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,2019-08-23 15:08:11,115.8,San Jose,California,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,6,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190340,190340,$ae_session,willwojt@gmail.com_6868,2020-03-07 12:11:54,56.5,Washington,District of Columbia,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,29807,40862.0
190341,190341,$ae_session,willwojt@gmail.com_6868,2020-03-07 12:14:44,32.9,Washington,District of Columbia,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,29808,170.0
190342,190342,$ae_session,willwojt@gmail.com_6868,2020-03-08 10:47:21,42.3,Washington,District of Columbia,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,29809,81157.0
190343,190343,$ae_session,willwojt@gmail.com_6868,2020-03-08 10:52:05,200.6,Washington,District of Columbia,US,-1,-1,...,NaN,NaN,US,1,0,0,0,0,29810,284.0


In [18]:
max(df['time_delta'])

18289482.0

In [19]:
def best_fit_slope(xs,ys):
    m = (((np.mean(xs)*np.mean(ys)) - np.mean(xs*ys)) /
         ((np.mean(xs)*np.mean(xs)) - np.mean(xs*xs)))
    return m

In [20]:
grouped = df.groupby('distinct_id', as_index=False).agg({'time' : 'count'})
grouped

,distinct_id,time
0,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,10
1,001210DC-54C8-43AD-B295-148F47818391,9
2,0025A8A4-2590-4ECA-8CE8-419D710AE46F,7
3,004C9C76-A530-422A-BA1B-667CDA12A08D,3
4,005144D7-BD3F-405B-9621-5FACA6B1AC04,1
...,...,...
2617,varun.murthy@gmail.com_5928,5
2618,virtrutest99@gmail.com_5848,8
2619,wandaandreu@gmail.com_5053,2
2620,willwojt@gmail.com_6868,98


In [30]:
def add_slope(df):
    grouped = df.groupby('distinct_id', as_index=False).agg({'time' : 'count'})
    grouped['slope'] = None
    for index, row in grouped.iterrows():
        x = df.loc[df['distinct_id'] == row['distinct_id']]
        length = np.arange(len(x))
        slope = best_fit_slope(length, x['time_delta'])
        grouped.loc[grouped['distinct_id'] == row['distinct_id'], 'slope'] = slope
    final = grouped.drop(columns='time', axis=1)
    return final

add_slope(df)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


,distinct_id,slope
0,000BA5B1-DBEF-414E-ACEC-1A2FCC2053DD,1113.36
1,001210DC-54C8-43AD-B295-148F47818391,-11716
2,0025A8A4-2590-4ECA-8CE8-419D710AE46F,615550
3,004C9C76-A530-422A-BA1B-667CDA12A08D,-44.5
4,005144D7-BD3F-405B-9621-5FACA6B1AC04,NaN
...,...,...
2617,varun.murthy@gmail.com_5928,-436340
2618,virtrutest99@gmail.com_5848,17.2857
2619,wandaandreu@gmail.com_5053,1.37164e+07
2620,willwojt@gmail.com_6868,-122.179
